In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow import keras
from scipy import sparse
import pandas as pd
import os
from google.colab import drive

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
os.chdir("/content/drive/MyDrive/lendingClubLoanData/notebooks")

In [4]:
! ls

Exploration.ipynb  Preprocessing.ipynb	Train_Evaluate.ipynb


In [5]:
Ytrain = pd.read_csv("../../dataset/preprocessed_data/Ytrain.csv")
Ytest = pd.read_csv("../../dataset/preprocessed_data/Ytest.csv")
Xtrain_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtrain_preprocessed.npz")
Xtest_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtest_preprocessed.npz")

In [6]:
# Creation du modele

model = Sequential()
model.add(Input((Xtrain_preprocessed.shape[1],)))
model.add(Dense(units = 100 , activation = "relu"))
model.add(Dense(units = 60))
model.add(BatchNormalization())
model.add(Dense(units = 30, activation = "relu"))
model.add(Dense(units = 1, activation = "sigmoid"))


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │     2,016,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         6,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 60)             │           240 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 30)             │         1,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,024,561 (7.72 MB)

 Trainable params: 2,024,441 (7.72 MB)

 Non-trainable params: 120 (480.00 B)

In [8]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [9]:
Ytrain.shape

(12138, 1)

In [10]:
history = model.fit(
    Xtrain_preprocessed, Ytrain,
    validation_data = (Xtest_preprocessed, Ytest),
    epochs = 2,
    batch_size = 32,
    verbose = True
)

Epoch 1/2
380/380 ━━━━━━━━━━━━━━━━━━━━ 34s 82ms/step - accuracy: 0.9735 - loss: 0.1056 - val_accuracy: 0.9942 - val_loss: 0.0291
Epoch 2/2
380/380 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9957 - loss: 0.0119 - val_accuracy: 0.9918 - val_loss: 0.0310


# Evaluation du modele

In [11]:
Xval_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xval_preprocessed.npz")
Yval = pd.read_csv("../../dataset/preprocessed_data/Yval.csv")

predictions = model.predict(Xval_preprocessed)

177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step


In [12]:
Yval

,loan_status
0,1
1,1
2,1
3,1
4,1
...,...
5659,1
5660,1
5661,1
5662,1


In [13]:
print(classification_report(Yval, predictions.round()))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95       540
           1       0.99      1.00      1.00      5124

    accuracy                           0.99      5664
   macro avg       0.99      0.96      0.97      5664
weighted avg       0.99      0.99      0.99      5664



In [14]:
print(confusion_matrix(Yval, predictions.round()))

[[ 493   47]
 [   1 5123]]


In [15]:
Yval.value_counts(normalize = True)

,proportion
loan_status,
1,0.904661
0,0.095339


In [16]:
model.save("../models/model1.keras")